In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# add your code here - consider creating a new cell for each section of code
ratings_by_user = df_ratings.groupby('user').count()
keep_users = ratings_by_user[ratings_by_user['rating'] >= 200]
ratings_by_book = df_ratings.groupby('isbn').count()
keep_books = ratings_by_book[ratings_by_book['rating'] >= 100]

In [5]:
# keep_users.sort_values('rating'), keep_books.sort_values('rating')

In [6]:
mask = (df_ratings['isbn'].isin(keep_books.index)) & (df_ratings['user'].isin(keep_users.index))
df_ratings_dropped = df_ratings[mask]
df_ratings_dropped.head()

,user,isbn,rating
1456,277427,002542730X,10.0
1469,277427,0060930535,0.0
1471,277427,0060934417,0.0
1474,277427,0061009059,9.0
1484,277427,0140067477,0.0


In [7]:
full_df = pd.merge(df_books, df_ratings_dropped, 
                   how='right', on='isbn',
                   )
full_df.head()

,isbn,title,author,user,rating
0,002542730X,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,277427,10.0
1,0060930535,The Poisonwood Bible: A Novel,Barbara Kingsolver,277427,0.0
2,0060934417,Bel Canto: A Novel,Ann Patchett,277427,0.0
3,0061009059,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,277427,9.0
4,0140067477,The Tao of Pooh,Benjamin Hoff,277427,0.0


In [8]:
pivot_table = full_df.pivot_table(index='title', columns='user', values='rating').fillna(0)
pivot_table

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Without Remorse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
k = 5
kNN = NearestNeighbors(n_neighbors=k+1, metric='cosine', algorithm='brute')
nbrs = kNN.fit(pivot_table)

In [10]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  distances, indices = nbrs.kneighbors(pivot_table.loc[[book], :])
  title_distance =  []
  for title, distance in zip(indices[0], distances[0]):
    if distance != 0:
      title_distance.append([pivot_table.index[title], distance])
  recommended_books = [book, 
                      title_distance[::-1]] 
  return recommended_books

In [11]:
get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['Catch 22', np.float32(0.7939835)],
  ['The Witching Hour (Lives of the Mayfair Witches)', np.float32(0.74486566)],
  ['Interview with the Vampire', np.float32(0.73450685)],
  ['The Tale of the Body Thief (Vampire Chronicles (Paperback))',
   np.float32(0.53763384)],
  ['The Vampire Lestat (Vampire Chronicles, Book II)',
   np.float32(0.51784116)]]]

In [12]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", np.float32(0.8016211)], ['The Weight of Water', np.float32(0.77085835)], ['The Surgeon', np.float32(0.7699411)], ['I Know This Much Is True', np.float32(0.7677075)], ['The Lovely Bones: A Novel', np.float32(0.7234864)]]]
You passed the challenge! 🎉🎉🎉🎉🎉
